<center><img src='https://urldefense.com/v3/__https://sedigitalylanzate.es/wp-content/uploads/2019/03/logoeoiok-e1552040050697.png__;!!MXQJGYEVCjwd!AaJpMIhjhRWcegQ2aKyvx1P9gaSA5oC7F7sVurPi34hiHVo6k8-Ordl8Z_gm-SCRMSMR5lA$ ' width="500"></center>

# Ejercicio de algoritmo de filtro colaborativo

El objetivo de este ejercicio es generar un algoritmo de filtro colaborativo con los datos del dataset beer_review. La información se corresponde con la reviews de la web https://urldefense.com/v3/__https://www.beeradvocate.com/__;!!MXQJGYEVCjwd!AaJpMIhjhRWcegQ2aKyvx1P9gaSA5oC7F7sVurPi34hiHVo6k8-Ordl8Z_gm-SCRXm04eLI$ 

<center><img src='https://urldefense.com/v3/__https://www.pngitem.com/pimgs/m/39-394080_beer-advocate-hd-png-download.png__;!!MXQJGYEVCjwd!AaJpMIhjhRWcegQ2aKyvx1P9gaSA5oC7F7sVurPi34hiHVo6k8-Ordl8Z_gm-SCRskeByAY$ ' width=600 ></center>

## Información

Este dataset consiste en revisiones de cerveza de Beeradvocate. Los datos abarcan un período de más de 10 años, que incluye todas ~ 1.5 millones de revisiones hasta noviembre de 2011. Cada revisión incluye calificaciones en términos de cinco "aspectos": apariencia, aroma, paladar, sabor e impresión general. Las revisiones incluyen información del producto y del usuario, seguida de cada una de estas cinco clasificaciones, y una revisión en formato texto. 

Para nuestro ejercicio vamos a usar una versión reducida de este dataset, que contiene las reviews de aquellos usuarios que han valorado al menos 50 cervezas.

## Pasos a seguir

1. Descarga la información de beeradvocate en una carpeta que se llame `datos_ejercicios/beeradvocate` y que se encuentre al mismo nivel que el notebook en el que vas a ejecutar el código.

2. Lee el fichero `beer_reviews_50.csv` y crea dos datasets:
    - uno con las reviews de los usuarios.
    - otro con la información de la cerveza (brewery, beer_name, beer_style)
    
3. Divide la información en los conjuntos de train y test.

4. De los distintos tipos de algoritmos que conocemos elige uno para generar la recomendación. Si creas más de uno explica el motivo de tu elección.


5. Crea una función que dado un usuario nos devuelva el top 10 de recomendaciones. 

6. Genera un html con los resultados de notebook file -> Export Notebook As... -> Export Notebook as HTML. Este html será el documento que tendrás que entregar para la evaluación del ejercicio.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import seaborn as sns

In [3]:
totalDB = pd.read_csv('/content/drive/MyDrive/datos_ejercicios/beeradvocate/beer_reviews_50.csv', low_memory=False)

In [4]:
totalDB.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,user_id
0,163,Amstel Brouwerij B. V.,1023302870,3.5,3.5,3.5,Mar-77,Light Lager,3.0,3.0,Amstel Light,3.50,436,1
1,395,Bluegrass Brewing Co. - East St. Matthew's,1055007422,4.5,4.0,4.5,Mar-77,American Pale Ale (APA),3.5,4.0,American Pale Ale,5.79,1122,1
2,395,Bluegrass Brewing Co. - East St. Matthew's,1055009536,5.0,3.5,4.5,Mar-77,English Porter,3.5,4.5,Dark Star Porter,5.60,1123,1
3,365,Pike Pub & Brewery,1052070806,5.0,4.5,4.5,Mar-77,American Stout,4.0,4.5,Pike Street XXXXX Stout,7.00,966,1
4,1199,Founders Brewing Company,1065491961,4.5,4.5,4.0,Mar-77,Scotch Ale / Wee Heavy,3.0,4.0,Founders Dirty Bastard,8.50,7463,1


In [5]:
reviews = totalDB[['user_id','review_profilename','review_overall','review_aroma','review_appearance','review_palate','review_taste','beer_abv']]

In [6]:
cervezas = totalDB[['beer_beerid','brewery_name','beer_style','beer_name']]

In [7]:
reviews.head()

,user_id,review_profilename,review_overall,review_aroma,review_appearance,review_palate,review_taste,beer_abv
0,1,Mar-77,3.5,3.5,3.5,3.0,3.0,3.50
1,1,Mar-77,4.5,4.0,4.5,3.5,4.0,5.79
2,1,Mar-77,5.0,3.5,4.5,3.5,4.5,5.60
3,1,Mar-77,5.0,4.5,4.5,4.0,4.5,7.00
4,1,Mar-77,4.5,4.5,4.0,3.0,4.0,8.50


In [9]:
cervezas.head()

,beer_beerid,brewery_name,beer_style,beer_name
0,436,Amstel Brouwerij B. V.,Light Lager,Amstel Light
1,1122,Bluegrass Brewing Co. - East St. Matthew's,American Pale Ale (APA),American Pale Ale
2,1123,Bluegrass Brewing Co. - East St. Matthew's,English Porter,Dark Star Porter
3,966,Pike Pub & Brewery,American Stout,Pike Street XXXXX Stout
4,7463,Founders Brewing Company,Scotch Ale / Wee Heavy,Founders Dirty Bastard


In [10]:
U_rating = totalDB[['user_id','beer_beerid','review_overall']] 

In [11]:
U_rating.head()

,user_id,beer_beerid,review_overall
0,1,436,3.5
1,1,1122,4.5
2,1,1123,5.0
3,1,966,5.0
4,1,7463,4.5


In [12]:
conteo_rating = (
    U_rating[['user_id','beer_beerid']]
    .groupby('user_id')
    .count().reset_index()
    .rename(columns={'beer_beerid': 'count'})
)

In [13]:
minimo_votos = conteo_rating['count'].quantile(0.70)

In [14]:
minimo_votos

147.0

In [15]:
lista_votos = conteo_rating[conteo_rating['count']>= minimo_votos].user_id.tolist()

In [16]:
U_rating = U_rating[U_rating.user_id.isin(lista_votos)]

In [17]:
from sklearn.model_selection import train_test_split


X = U_rating.copy()
y = U_rating['user_id']

In [18]:
train_test_split

<function sklearn.model_selection._split.train_test_split>

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
mean_squared_error

<function sklearn.metrics._regression.mean_squared_error>

In [22]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [23]:
def baseline(usuario,cerveza):
    return 3.0

In [24]:
pares = zip(X_test['user_id'], X_test['beer_beerid'])

In [25]:
def calculo_rmse(modelo):
    
    #Creamos una tupla de usuarios-cervezas del conjunto de test
    pares = zip(X_test['user_id'], X_test['beer_beerid'])
    
    #Hacemos una predicción para cada tupla
    y_pred = np.array([modelo(user_id, beer_beerid) for (user_id, beer_beerid) in pares])
    
    #Sacamos el dato real
    y_true = np.array(X_test['review_overall'])
    
    #devolvemos el valor de rmse
    return rmse(y_true, y_pred)

    

In [26]:
calculo_rmse(baseline)

1.0500646154202502

In [27]:
usuario_ejemplo = int(reviews['user_id'].sample(n=1, random_state=1))

In [28]:
usuario_ejemplo

808

In [29]:
puntuaciones_ejemplo = (
    U_rating[U_rating.user_id == usuario_ejemplo]
    .sort_values("review_overall", ascending=False)
    .merge(
        cervezas[["beer_beerid", "beer_name"]],
        on='beer_beerid'
    )
)

In [30]:
puntuacion_ejemplo = puntuaciones_ejemplo.drop_duplicates()

In [32]:
puntuacion_ejemplo.head()

,user_id,beer_beerid,review_overall,beer_name
0,808,771,5.0,Entire Butt English Porter
122,808,473,5.0,Dunster Street Amber
124,808,473,4.0,Dunster Street Amber
126,808,6171,5.0,Best Bitter
129,808,666,5.0,Hacker-Pschorr Dunkle Weisse


In [33]:
pd.options.display.max_rows = puntuacion_ejemplo.shape[0]

In [34]:
puntuacion_ejemplo.sort_values(["review_overall", "beer_beerid"], ascending=False).head(10)

,user_id,beer_beerid,review_overall,beer_name
316,808,39201,5.0,Hopworks IPA
411,808,18595,5.0,Irish Stout
3314,808,8784,5.0,Hirsch Bavarian-Weissbier Hefeweisse
126,808,6171,5.0,Best Bitter
2600,808,4064,5.0,Brakspear Bitter
3368,808,1871,5.0,Dobbelen Bruinen
418,808,1818,5.0,Waas Klokbier
2640,808,1658,5.0,Big Bear Black Stout
0,808,771,5.0,Entire Butt English Porter
129,808,666,5.0,Hacker-Pschorr Dunkle Weisse


In [35]:
vistas_ejemplo = puntuacion_ejemplo.beer_beerid.tolist()

In [36]:
matriz = X_train.pivot_table(values='review_overall', index='beer_beerid', columns='user_id')

In [37]:
matriz.head()

user_id,1,8,10,13,16,17,18,20,21,25,29,30,33,34,35,38,39,40,44,45,46,50,51,55,65,66,68,71,73,74,75,78,79,82,84,85,86,92,98,104,...,1696,1698,1706,1707,1708,1712,1724,1725,1726,1731,1734,1741,1747,1750,1751,1756,1757,1767,1768,1769,1779,1782,1783,1784,1785,1786,1787,1788,1790,1796,1799,1803,1806,1810,1812,1814,1817,1824,1827,1828
beer_beerid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN
13,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,1.5,NaN


In [50]:
def fc_media(usuario,cerveza):
    
    #Comprobamos que la película esté en la sparse-matrix
    if cerveza in matriz:
        #Calculamos la media
        media = matriz[cerveza].mean()
    
    else:
        #En caso de no tenerla devolvemos un 3
        media = 3.0
    
    return media

In [51]:
calculo_rmse(fc_media)

1.039023735446685

In [75]:
lista_cervezas = cervezas.beer_beerid.drop_duplicates()

In [85]:
lista_cervezas.head

<bound method NDFrame.head of 0           436
1          1122
2          1123
3           966
4          7463
          ...  
256009    72794
256199    75896
256200    76092
256202    76093
256204    75874
Name: beer_beerid, Length: 13693, dtype: int64>

In [ ]:
recomendaciones_media = []
for i in cervezas.beer_beerid.to_list():
    
    recomendaciones_media.append([i, fc_media(usuario_ejemplo, i)])
    
    recos =(
        pd.DataFrame(recomendaciones_media,
                     columns=["beer_beerid", "review_overall"]
                    )
        .sort_values("review_overall", ascending=False)
        .merge(
            cervezas[["beer_beerid", "beer_name"]],
            on='beer_beerid'
        )
    
    )
recos = recos[~recos.beer_beerid.isin(vistas_ejemplo)].head(20)

In [110]:
recos

,beer_beerid,review_overall,beer_name
0,1664,4.169872,Kennebunk Porter
1,1664,4.169872,Kennebunk Porter
2,1664,4.169872,Kennebunk Porter
3,1664,4.169872,Kennebunk Porter
4,1664,4.169872,Kennebunk Porter
5,1664,4.169872,Kennebunk Porter
6,1664,4.169872,Kennebunk Porter
7,1664,4.169872,Kennebunk Porter
8,1664,4.169872,Kennebunk Porter
9,1664,4.169872,Kennebunk Porter
